# Alive & Thrive
## Cambodia Data: Children Files - Tabulation
## Prepared by Aaron Wise; aaron@a3di.dev
### Version: 15 November 2022

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)

import statsmodels.api as sm

In [2]:
from tabulate_vars import (
    read_csv_file,
    concatenate_dfs,
    save_combined,
    create_bivariate_var_dep,
    extract_regression_params
)

from aw_analytics import mean_wt, output_mean_table

### Create and save combined children's file

In [3]:
# Set parameters
country = 'KHM'
recode = 'children'

# -------------------------------------------------------------------
year_list = ['2000', '2005', '2010', '2014'] 
# -------------------------------------------------------------------


# Read in files, store dfs in list
list_of_dfs = [read_csv_file(country, recode, year, file_type='working') for year in year_list]

# Combine the dfs
df = concatenate_dfs(list_of_dfs)

# Save and export
save_combined(df, country, recode)

The file -- KHM_children_2000_working.csv -- has the following shape: Rows: 6079; Columns: 27
The file -- KHM_children_2005_working.csv -- has the following shape: Rows: 6142; Columns: 30
pnc_A, pnc_B and pnc_C not in the files
The file -- KHM_children_2010_working.csv -- has the following shape: Rows: 6448; Columns: 43
pnc_A, pnc_B and pnc_C not in the files
The file -- KHM_children_2014_working.csv -- has the following shape: Rows: 5901; Columns: 43


### Run Descriptive stats

In [4]:
## TOTAL DATASET

# Set parameters
if recode == 'children':
    vars = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch']

elif recode == 'anthro':
    vars = ['stunting_ch', 'wasting_ch', 'overweight_ch']

else:
    print("Invalid recode!!!")

ind_vars = ['Total', 'sex_ch', 'residence', 'region', 'mother_edu', 'wealth_q', 'elderly_hoh', 'sex_hoh']

wt = 'chweight'

# Run for loop

for year in year_list:

    df = read_csv_file(country, recode, year, file_type='working')

    out_fn = country + "_" + recode + "_" + year + ".xlsx"
    path = Path.cwd() / 'output' / 'frequencies' / recode / out_fn
    
    # Instantiate object
    xlwriter = pd.ExcelWriter(path)
    
    for var in vars:

        try:
            # df = df.dropna(subset=[var])
            output = output_mean_table(df.dropna(subset=[var]), var, ind_vars, wt)
            sheet_name = str(var) + '_weighted'
            output.to_excel(xlwriter, sheet_name=sheet_name)
        
        except:
            print(f"Warning: Unable to create sheet for var {var}")

    xlwriter.close()

The file -- KHM_children_2000_working.csv -- has the following shape: Rows: 6079; Columns: 27
The file -- KHM_children_2005_working.csv -- has the following shape: Rows: 6142; Columns: 30
pnc_A, pnc_B and pnc_C not in the files
The file -- KHM_children_2010_working.csv -- has the following shape: Rows: 6448; Columns: 43
pnc_A, pnc_B and pnc_C not in the files
The file -- KHM_children_2014_working.csv -- has the following shape: Rows: 5901; Columns: 43


### Run Bivariate stats

In [5]:
# Read in combined file
df = read_csv_file(country, recode, file_type='combined')

# Create updated bivariate variables (mother_edu_biv, eth_hoh_biv)
df = create_bivariate_var_dep(df, country)

The file -- KHM_children_combined.csv -- has the following shape: Rows: 24570; Columns: 43


In [6]:
# Set parameters
# Set parameters
if recode == 'children':
    var_dep_list = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch']

elif recode == 'anthro':
    var_dep_list = ['stunting_ch', 'wasting_ch', 'overweight_ch']

else:
    print("Invalid recode!!!")

ind_var_list = ['sex_ch', 'residence', 'region', 'mother_edu_biv', 'wealth_q', 'elderly_hoh', 'sex_hoh']

# Run bivariate (extract WLS regression params)
output = {var_dep: pd.DataFrame({ind_var: extract_regression_params(df, var_dep, ind_var, recode) for ind_var in ind_var_list}) for var_dep in var_dep_list}

var_dep and ind_var are: 
 ['excl_bf'], ['sex_ch']
var_dep and ind_var are: 
 ['excl_bf'], ['residence']
var_dep and ind_var are: 
 ['excl_bf'], ['region']
var_dep and ind_var are: 
 ['excl_bf'], ['mother_edu_biv']
var_dep and ind_var are: 
 ['excl_bf'], ['wealth_q']
var_dep and ind_var are: 
 ['excl_bf'], ['elderly_hoh']
var_dep and ind_var are: 
 ['excl_bf'], ['sex_hoh']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['sex_ch']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['residence']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['region']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['mother_edu_biv']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['wealth_q']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['elderly_hoh']
var_dep and ind_var are: 
 ['cont_1223_bf'], ['sex_hoh']
var_dep and ind_var are: 
 ['mdd_ch'], ['sex_ch']
var_dep and ind_var are: 
 ['mdd_ch'], ['residence']
var_dep and ind_var are: 
 ['mdd_ch'], ['region']
var_dep and ind_var are: 
 ['mdd_ch'], ['mother_edu_biv']
va

In [7]:
# Generate csv of outputs
for var in var_dep_list:
    fn = var + '.csv'
    output[var].transpose().to_csv(f"./output/bivariate/{recode}/{fn}")

#### Get bivariate differences

In [8]:
df = read_csv_file(country, recode, file_type='combined')

df = create_bivariate_var_dep(df, country)

# Set parameters
if recode == 'children':
    var_dep_list = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch']

elif recode == 'anthro':
    var_dep_list = ['stunting_ch', 'wasting_ch', 'overweight_ch']

else:
    print("Invalid recode!!!")

ind_var_list = ['sex_ch', 'residence', 'region', 'mother_edu_biv', 'wealth_q',  'elderly_hoh', 'sex_hoh']


for var in var_dep_list:

    out_fn = var + "_bivariate_diff" + ".xlsx"
    path = Path.cwd() / 'output' / 'bivariate' / recode / out_fn

    # Instantiate object
    xlwriter = pd.ExcelWriter(path)

    for ind_var in ind_var_list:

        output = df.groupby(['Year', ind_var]).apply(mean_wt, var, wt=wt).unstack().transpose()
        
        sheet_name = str(ind_var) + '_weighted'
        output.to_excel(xlwriter, sheet_name=sheet_name)

    xlwriter.close()

The file -- KHM_children_combined.csv -- has the following shape: Rows: 24570; Columns: 43
